In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqlalchemy import create_engine
import matplotlib as mpl
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
import time
import sys
sys.path.insert(0, 'XXXX-XXXX')
from calc_elo import *
%matplotlib inline

In [5]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path, timeout=10)  

In [6]:
def get_team_rating(team_id):
    path = r"XXXX-XXXX"
    con = sqlite3.connect(path)
    cur = con.cursor()
    df = pd.read_sql_query("SELECT *  FROM teamRatings AS tr WHERE team_id = (?)", con, params=(team_id,))
    elo = float(df.loc[0]["elo"])
    attack = float(df.loc[0]["attack"])
    defence = float(df.loc[0]["defence"])
    con.close()
    return elo, attack, defence


def team_values_before_fixture(team_id ,timestamp):
    team_ratings = pd.read_sql_query("SELECT * FROM teamRatings WHERE team_id = (?) AND timestamp < (?) ORDER BY timestamp DESC LIMIT 1", con, params=(team_id, timestamp))
    if(len(team_ratings)==0):
        team_ratings = pd.read_sql_query("SELECT * FROM teamRatings WHERE team_id = (?) AND timestamp = (?) ORDER BY timestamp DESC LIMIT 1", con, params=(team_id, timestamp))
    elo = float(team_ratings.loc[0]["elo"])
    attack = float(team_ratings.loc[0]["attack"])
    defence = float(team_ratings.loc[0]["defence"])
    return elo, attack, defence

def len_fixtures():
    path = r"XXXX-XXXX"
    con = sqlite3.connect(path)
    cur = con.cursor()
    all = cur.execute('SELECT * FROM fixtures AS f').fetchall()
    con.close()
    return len(all)

In [7]:
df = pd.read_sql_query("SELECT  fixture_id, timestamp ,home_id FROM fixtures ", con)
df.insert(2, 'team_id', df.pop('home_id'))
df['elo'] = float(1500)
df['attack'] = float(0)
df['defence'] = float(0)
display(df)

,fixture_id,timestamp,team_id,elo,attack,defence
0,65,1533927600,33,1500.0,0.0,0.0
1,66,1533987000,34,1500.0,0.0,0.0
2,67,1533996000,35,1500.0,0.0,0.0
3,68,1533996000,36,1500.0,0.0,0.0
4,69,1533996000,37,1500.0,0.0,0.0
...,...,...,...,...,...,...
14666,705637,1622390400,83,1500.0,0.0,0.0
14667,705680,1622046600,192,1500.0,0.0,0.0
14668,705681,1622304000,191,1500.0,0.0,0.0
14669,705682,1622061900,240,1500.0,0.0,0.0


In [8]:
df_a = pd.read_sql_query("SELECT  fixture_id, timestamp ,away_id FROM fixtures ", con)
df_a.insert(2, 'team_id', df_a.pop('away_id'))
df_a['elo'] = float(1500)
df_a['attack'] = float(0)
df_a['defence'] = float(0)
display(df_a)

,fixture_id,timestamp,team_id,elo,attack,defence
0,65,1533927600,46,1500.0,0.0,0.0
1,66,1533987000,47,1500.0,0.0,0.0
2,67,1533996000,43,1500.0,0.0,0.0
3,68,1533996000,52,1500.0,0.0,0.0
4,69,1533996000,49,1500.0,0.0,0.0
...,...,...,...,...,...,...
14666,705637,1622390400,96,1500.0,0.0,0.0
14667,705680,1622046600,191,1500.0,0.0,0.0
14668,705681,1622304000,192,1500.0,0.0,0.0
14669,705682,1622061900,226,1500.0,0.0,0.0


In [9]:
team_ratings = df.append(df_a)
display(team_ratings)

,fixture_id,timestamp,team_id,elo,attack,defence
0,65,1533927600,33,1500.0,0.0,0.0
1,66,1533987000,34,1500.0,0.0,0.0
2,67,1533996000,35,1500.0,0.0,0.0
3,68,1533996000,36,1500.0,0.0,0.0
4,69,1533996000,37,1500.0,0.0,0.0
...,...,...,...,...,...,...
14666,705637,1622390400,96,1500.0,0.0,0.0
14667,705680,1622046600,191,1500.0,0.0,0.0
14668,705681,1622304000,192,1500.0,0.0,0.0
14669,705682,1622061900,226,1500.0,0.0,0.0


In [ ]:
#CREAT TeamRatings_table
engine = create_engine(r'sqlite:///XXXX-XXXX', echo=False)
team_ratings.to_sql('teamRatings', con=engine, if_exists='fail') #if_exists='append'

In [11]:
team_ratings = pd.read_sql_query("SELECT * FROM teamRatings WHERE team_id = 34 ORDER BY timestamp DESC", con)
display(team_ratings)

,index,fixture_id,timestamp,team_id,elo,attack,defence
0,13582,592868,1621782000,34,1497.164844,1.495549,-1.665210
1,13576,592862,1621443600,34,1486.302062,1.440315,-1.811980
2,13566,592852,1621018800,34,1478.948349,1.499525,-1.965757
3,13554,592840,1620414000,34,1481.405406,1.313876,-1.760613
4,13546,592832,1619960400,34,1459.337923,1.017757,-1.738447
...,...,...,...,...,...,...,...
185,8829,192346,1442257200,34,1462.886578,0.123480,-0.564590
186,8810,192327,1440848700,34,1476.307102,0.147772,-0.416119
187,8800,192317,1440243900,34,1486.003987,0.166563,-0.346687
188,8793,192310,1439647200,34,1485.000000,0.180000,-0.380000


In [12]:
team_values_before_fixture(34, 1620414000)

(1459.33792314527, 1.0177570343732016, -1.7384465280544363)

In [13]:
print(team_values_before_fixture(34, 1621782000))

(1486.3020616759354, 1.4403150008638035, -1.8119802556700002)


In [14]:
def calculate_team_ratings_after_fixture(): 
     for i in range(0, len_fixtures()):
        start = time.time()
        fixture_id, home_id, away_id, timestamp = get_fixture_home_away_id_timestamp(i)
        home_goals, away_goals, result = get_fixture_goals_result(fixture_id)
        goal_diff = calc_goal_difference(home_goals, away_goals)
        print(home_goals)
        print(away_goals)
        print(result)
        print("DIFF", calc_goal_difference(home_goals, away_goals))
        home_elo_alt, home_att_alt, home_def_alt = team_values_before_fixture(home_id, timestamp)
        away_elo_alt, away_att_alt, away_def_alt = team_values_before_fixture(away_id, timestamp)
        

        if None not in (home_elo_alt, home_att_alt, home_def_alt, away_elo_alt, away_att_alt, away_def_alt, home_goals, away_goals, result):
            
            home_elo = update_elo(home_elo_alt,  away_elo_alt, result, goal_diff, True)
            home_att = update_attack(home_att_alt, home_goals, away_def_alt)
            home_def = update_defence(home_def_alt, away_goals, away_att_alt)
            away_elo = update_elo(away_elo_alt,  home_elo_alt, result, goal_diff, False)
            away_att = update_attack(away_att_alt, away_goals, home_def_alt)
            away_def = update_defence(away_def_alt, home_goals, home_att_alt)
            elapsed_time_fl = (time.time() - start)
            print(str(elapsed_time_fl)+ " time")
            print("after")
            print("HOME",home_elo, home_att, home_def)
            print("AWAY",away_elo, away_att, away_def)
            print("fixture", i)
            print("")
            
            #add updated rating
            update_team_rating(home_id,home_elo, home_att, home_def, fixture_id)
            update_team_rating(away_id, away_elo, away_att, away_def, fixture_id)

In [15]:
high_elos = pd.read_sql_query("SELECT * FROM teamRatings ORDER BY elo DESC", con)
high_elos.insert(2, 'home_id', high_elos.pop('team_id'))
display(high_elos)

,index,fixture_id,home_id,timestamp,elo,attack,defence
0,7597,157280,40,1582574400,1918.368941,2.659446,-0.298063
1,7588,157271,40,1581787800,1916.974632,2.601538,-0.137389
2,7576,157259,40,1580569200,1916.185295,2.752337,-0.167222
3,7511,157194,40,1580327100,1912.953015,2.585022,-0.220586
4,7567,157250,40,1579809600,1910.706588,2.625336,-0.270903
...,...,...,...,...,...,...,...
29337,338,403,37,1555155000,1303.480154,0.293693,-2.437800
29338,1682,9513,534,1526747400,1303.427501,0.295450,-1.903432
29339,7343,157026,66,1566050400,1301.276333,0.496507,-2.514987
29340,345,410,37,1555768800,1298.795015,0.332936,-2.418041


In [16]:
fixtures = pd.read_sql_query("SELECT fixture_id, home_id, home_name, away_id, away_name  FROM fixtures ORDER BY fixture_id DESC", con)
display(fixtures)

,fixture_id,home_id,home_name,away_id,away_name
0,705683,226,Rio Ave,240,Arouca
1,705682,240,Arouca,226,Rio Ave
2,705681,191,Holstein Kiel,192,FC Koln
3,705680,192,FC Koln,191,Holstein Kiel
4,705637,83,Nantes,96,Toulouse
...,...,...,...,...,...
14666,69,37,Huddersfield,49,Chelsea
14667,68,36,Fulham,52,Crystal Palace
14668,67,35,Bournemouth,43,Cardiff
14669,66,34,Newcastle,47,Tottenham


In [17]:
home_elos = pd.merge(fixtures, high_elos, how='inner', on=['fixture_id', 'home_id'])
display(home_elos)

,fixture_id,home_id,home_name,away_id,away_name,index,timestamp,elo,attack,defence
0,705683,226,Rio Ave,240,Arouca,14670,1622397600,1460.850376,0.413921,-1.469398
1,705682,240,Arouca,226,Rio Ave,14669,1622061900,1439.388304,1.230076,-1.947105
2,705681,191,Holstein Kiel,192,FC Koln,14668,1622304000,1484.301317,0.102462,-0.454465
3,705680,192,FC Koln,191,Holstein Kiel,14667,1622046600,1416.686538,0.910708,-1.850755
4,705637,83,Nantes,96,Toulouse,14666,1622390400,1470.061449,1.534271,-1.242051
...,...,...,...,...,...,...,...,...,...,...
14666,69,37,Huddersfield,49,Chelsea,4,1533996000,1417.997221,0.317305,-1.365328
14667,68,36,Fulham,52,Crystal Palace,3,1533996000,1485.130306,-0.030110,-0.157769
14668,67,35,Bournemouth,43,Cardiff,2,1533996000,1493.602505,1.411180,-1.685251
14669,66,34,Newcastle,47,Tottenham,1,1533987000,1480.513543,1.074759,-1.065429


In [18]:
test = pd.read_sql_query("SELECT * FROM teamRatings WHERE fixture_id = 705680 AND team_id = 192 ORDER BY fixture_id DESC", con)
display(test)

,index,fixture_id,timestamp,team_id,elo,attack,defence
0,14667,705680,1622046600,192,1416.686538,0.910708,-1.850755


In [19]:
test = pd.read_sql_query("SELECT * FROM fixtures WHERE fixture_id = 192357  ORDER BY fixture_id DESC", con)
display(test)

,fixture_id,referee,timestamp,league_id,league_name,season,league_round,result,home_id,home_name,home_goals,away_id,away_name,away_goals
0,192357,"Mark Clattenburg, England",1443267900,39,Premier League,2015,Regular Season - 7,0.0,47,Tottenham,4,50,Manchester City,1


In [ ]:
def calculate_team_ratings_before_fixture_home(): #len_fixtures()
        data = []
        for i in range(0, len_fixtures()):
                start = time.time()
                print(i)
                fixture_id, home_id, away_id, timestamp = get_fixture_home_away_id_timestamp(i)
                elo, att, defe = team_values_before_fixture(home_id, timestamp)
                data.append([fixture_id, home_id, elo, att, defe])
                # test = pd.read_sql_query("SELECT * FROM teamRatings WHERE fixture_id = (?) AND team_id = (?) ORDER BY fixture_id DESC", con, params=(fixture_id, home_id))
                # display(test)

        return data   


data = calculate_team_ratings_before_fixture_home()
home_elo = pd.DataFrame(data, columns=['fixture_id', 'home_id', 'h_elo', 'h_attack', 'h_defence'])
display(home_elo)


In [26]:
def calculate_team_ratings_before_fixture_away(): #len_fixtures()
        data = []
        for i in range(0, len_fixtures()):
                fixture_id, home_id, away_id, timestamp = get_fixture_home_away_id_timestamp(i)
                elo, att, defe = team_values_before_fixture(away_id, timestamp)
                data.append([fixture_id, away_id, elo, att, defe])
                # test = pd.read_sql_query("SELECT * FROM teamRatings WHERE fixture_id = (?) AND team_id = (?) ORDER BY fixture_id DESC", con, params=(fixture_id, away_id))
                # display(test)

        return data   


data = calculate_team_ratings_before_fixture_away()
away_elo = pd.DataFrame(data, columns=['fixture_id', 'away_id', 'a_elo', 'a_attack', 'a_defence'])
display(away_elo)

,fixture_id,away_id,a_elo,a_attack,a_defence
0,192297,47,1490.000000,0.000000,-0.100000
1,192298,66,1510.000000,0.100000,0.000000
2,192299,52,1515.000000,0.300000,-0.100000
3,192300,746,1485.000000,0.200000,-0.400000
4,192301,38,1500.000000,0.200000,-0.200000
...,...,...,...,...,...
14666,605441,533,1612.400417,2.063742,-1.195641
14667,605443,530,1748.579661,1.695671,-0.504819
14668,605444,531,1548.029090,1.098529,-0.929842
14669,605437,546,1478.396816,0.560318,-1.082957


In [27]:
result = pd.merge(home_elo, away_elo, how='inner', on=['fixture_id'])
display(result)

,fixture_id,home_id,h_elo,h_attack,h_defence,away_id,a_elo,a_attack,a_defence
0,192297,33,1510.000000,0.100000,0.000000,47,1490.000000,0.000000,-0.100000
1,192298,35,1490.000000,0.000000,-0.100000,66,1510.000000,0.100000,0.000000
2,192299,71,1485.000000,0.100000,-0.300000,52,1515.000000,0.300000,-0.100000
3,192300,46,1515.000000,0.400000,-0.200000,746,1485.000000,0.200000,-0.400000
4,192301,45,1500.000000,0.200000,-0.200000,38,1500.000000,0.200000,-0.200000
...,...,...,...,...,...,...,...,...,...
14666,605441,541,1788.638758,2.013614,-0.416795,533,1612.400417,2.063742,-1.195641
14667,605443,720,1414.838818,0.692423,-1.753869,530,1748.579661,1.695671,-0.504819
14668,605444,797,1427.352803,0.694785,-1.534431,531,1548.029090,1.098529,-0.929842
14669,605437,715,1464.711878,1.396360,-2.123732,546,1478.396816,0.560318,-1.082957


In [28]:

engine = create_engine(r'XXXX-XXXX', echo=False,  connect_args={'timeout': 15})
result.to_sql('preFixtureTeamRatings', con=engine, if_exists='fail') #if_exists='append'
test = pd.read_sql_query("SELECT  * FROM preFixtureTeamRatings;", con)
display(test)


,index,fixture_id,home_id,h_elo,h_attack,h_defence,away_id,a_elo,a_attack,a_defence
0,0,192297,33,1510.000000,0.100000,0.000000,47,1490.000000,0.000000,-0.100000
1,1,192298,35,1490.000000,0.000000,-0.100000,66,1510.000000,0.100000,0.000000
2,2,192299,71,1485.000000,0.100000,-0.300000,52,1515.000000,0.300000,-0.100000
3,3,192300,46,1515.000000,0.400000,-0.200000,746,1485.000000,0.200000,-0.400000
4,4,192301,45,1500.000000,0.200000,-0.200000,38,1500.000000,0.200000,-0.200000
...,...,...,...,...,...,...,...,...,...,...
14666,14666,605441,541,1788.638758,2.013614,-0.416795,533,1612.400417,2.063742,-1.195641
14667,14667,605443,720,1414.838818,0.692423,-1.753869,530,1748.579661,1.695671,-0.504819
14668,14668,605444,797,1427.352803,0.694785,-1.534431,531,1548.029090,1.098529,-0.929842
14669,14669,605437,715,1464.711878,1.396360,-2.123732,546,1478.396816,0.560318,-1.082957


In [32]:
test = pd.read_sql_query("SELECT * FROM teamRatings WHERE fixture_id = (?) AND team_id = (?) ORDER BY fixture_id DESC", con, params=(605441, 533))
display(test)

,index,fixture_id,timestamp,team_id,elo,attack,defence
0,14272,605441,1621699200,533,1607.078451,1.998541,-1.255628


In [33]:
test = pd.read_sql_query("SELECT * FROM preFixtureTeamRatings WHERE fixture_id = (?) AND away_id = (?) ORDER BY fixture_id DESC", con, params=(605441, 533))
display(test)

,index,fixture_id,home_id,h_elo,h_attack,h_defence,away_id,a_elo,a_attack,a_defence
0,14666,605441,541,1788.638758,2.013614,-0.416795,533,1612.400417,2.063742,-1.195641


In [35]:
test = pd.read_sql_query("SELECT * FROM fixtures WHERE fixture_id = (?)  ORDER BY fixture_id DESC", con, params=(605441,))
display(test)

,fixture_id,referee,timestamp,league_id,league_name,season,league_round,result,home_id,home_name,home_goals,away_id,away_name,away_goals
0,605441,"Jose Luis Munuera Montero, Spain",1621699200,140,La Liga,2020,Regular Season - 38,0.0,541,Real Madrid,2,533,Villarreal,1


In [36]:
con.close()